In [2]:
from sage.rings.finite_rings.hom_finite_field import FiniteFieldHomomorphism_generic

In [3]:
def get_defining_set(n, pol, field):
    #defining_set = [field(0)]
    defining_set = []
    
    while len(defining_set) != n:
        aux = field.random_element()
        if pol(aux) != 0 and aux not in defining_set:
            defining_set = defining_set + [aux]
    
    return defining_set

In [114]:
from sage.coding.linear_code import AbstractLinearCode
from sage.coding.encoder import Encoder
from sage.coding.decoder import Decoder

class Goppa(AbstractLinearCode):
    r"""
    Implementation of Goppa codes.
    
    INPUT:
    - ``field`` -- finite field on which `self` is defined.
    - ``generating_pol`` -- a monic polynomial with coefficients in
    a finite field `\GF{p^m}` extending from `field`.
    
    - ``defining_set`` -- tuple of n distinct elements of `\GF{p^m}`
    that are not roots of `generating_pol`
    """
    def __init__(self, defining_set, generating_pol, field):
        """
        Initialize.
        """
        if not generating_pol.is_monic():
            raise ValueError("ERROR. Generating polynomial isn't monic")
        
        for gamma in defining_set:
            if generating_pol(gamma) == 0:
                raise ValueError("ERROR. Defining elements are roots of generating polynomial")
        
        self._field = field
        self._field_L = generating_pol.base_ring()

        if (not self._field.is_field() or not self._field.is_finite()):
            raise ValueError("ERROR. Generating polynomial isn't definied over a finite field")
        
        self._length = len(defining_set)
        self._generating_pol = generating_pol
        self._defining_set = defining_set
        
        super(Goppa, self).__init__(self._field, self._length, "GoppaEncoder", "GoppaDecoder")
    
    def _repr_(self):
        """
        Representation of a Goppa code.
        """
        return "[{}, {}] Goppa code".format(self._length, self.get_dimension())
    
    def get_generating_pol(self):
        """
        Return the generating polynomial of ``self``.
        """ 
        return self._generating_pol
    
    def get_defining_set(self):
        """
        Return the defining set of ``self``.
        """ 
        return self._defining_set
    
    def get_parity_pol(self):
        """
        Return the parity polynomial of ``self``.
        """
        parity_pol = list()
        
        for elem in self._defining_set:
            parity_pol.append((self._generating_pol.parent().gen() - elem).inverse_mod(self._generating_pol))
        
        return parity_pol
    
    def get_parity_check_matrix(self):
        """
        Return a parity check matrix of ``self``.
        """
        V, from_V, to_V = self._field_L.vector_space(self._field, map = True)

        parity = self.get_parity_pol()

        vector_L = []
        for i in range(len(parity)):
            vector_L = vector_L + parity[i].list()

        vector_F = []
        for i in range(len(vector_L)):
            vector_F = vector_F + to_V(vector_L[i]).list()

        matriz_F = matrix(self._length, vector_F)
        matriz_F = matriz_F.T
        
        return matriz_F
    
    def get_generator_matrix(self):
        """
        Return a generador matrix of ``self``.
        """
        H = self.get_parity_check_matrix()
        G = transpose(H).left_kernel().basis_matrix()
        
        return G
    
    def get_dimension(self):
        """
        Return the dimension of the code.
        """
        
        return rank(self.get_generator_matrix())
    
class GoppaEncoder(Encoder):
    r"""
    Encoder for Goppa codes
    
    INPUT:
    - ``code`` -- code associated with the encoder
    """
    def __init__(self, code):
        """
        Initialize.
        """
        super(GoppaEncoder, self).__init__(code)
        
    def _repr_(self):
        """
        Representation of a encoder for a Goppa code
        """
        return "Encoder for {}".format(self.code())
    
    def get_generator_matrix(self):
        """
        Return a generador matrix of the code
        """
        return self.code().get_generator_matrix()
    
    def encode (self, m):
        """
        Return a codeword
        
        INPUT:
        - ``m``: a vector to encode
        """
        return m * self.get_generator_matrix()
    
Goppa._registered_encoders["GoppaEncoder"] = GoppaEncoder

class GoppaDecoder(Decoder):
    r"""
    Decoder for Goppa codes
    
    INPUT:
    - ``code``: code associated with the decoder
    """
    
    def __init__(self, code):
        """
        Initialize
        """
        super(GoppaDecoder, self).__init__(code, code.ambient_space(), "GoppaDecoder")
                
        self._generating_pol = self.code().get_generating_pol()
        self._defining_set = self.code().get_defining_set()
    
    def _repr_(self):
        """
        Representation of a decoder for a Goppa code
        """
        return "Decoder for {}".format(self.code())
    
    def get_syndrome(self, c):
        """
        Return the syndrome polynomial
        
        INPUT:
        - ``c``: a element of the input space of ``self``.
        """
        field = self.code()._field
        field_L = self.code()._field_L
        
        embFL = FiniteFieldHomomorphism_generic(Hom(field,field_L))
        
        h = self.code().get_parity_pol()
        
        syndrome = 0
        
        for i in range(len(h)):    
            syndrome = syndrome + embFL(c[i])*h[i]
            
        return syndrome
    
    def get_generating_pol(self):
        """
        Return the generating polynomial
        """
        return self._generating_pol
    
    def decode_to_code(self, word):
        r"""
        Corrects the errors in ``word`` and returns a codeword.
        INPUT:
        - ``word`` -- a codeword of ``self``
        """
        i = 1
        field = self.code()._field
        field_L = self.code()._field_L
        
        embFL = FiniteFieldHomomorphism_generic(Hom(field,field_L))
        secLF = embFL.section()

        # Step 1
        S = self.get_syndrome(word)
        
        if S == 0:
            return word

        # Step 2
        r_prev = self.get_generating_pol()
        t = floor(self.get_generating_pol().degree()/2)
        r_i = S
        U_prev = 0
        U_i = 1
        
        # Steps 3 and 4
        while r_i.degree() >= t:
            (q, r) = r_prev.quo_rem(r_i)
            aux_r_i = r_i
            aux_U_i = U_i
            r_i = r
            U_i = q * U_i + U_prev
            r_prev = aux_r_i
            U_prev = aux_U_i  
            i += 1
            
        # Step 5
        # make sigma monic     
        sigma = U_i/U_i.coefficients()[-1]
        eta = (-1)^i * r_i / U_i.coefficients()[-1]
        
        # roots of sigma are the locations of the errors
        roots_loc = []
        
        for root in sigma.roots():
            roots_loc = roots_loc + [self._defining_set.index(root[0])]
        
        error = [0] * len(self._defining_set)
        x = self.get_generating_pol().parent().gen()
        sigma_diff = sigma.diff(x)
        
        for i in range(0, len(sigma.roots())):
            error[roots_loc[i]] = secLF(eta.subs(x=sigma.roots()[i][0])/(sigma_diff.subs(x=sigma.roots()[i][0])))
        
        x = word - vector(self.code()._field, error)
        
        return x

    def decode_to_message(self, word):
        word = self.decode_to_code(word)
        return self.code().get_parity_check_matrix().solve_left(word)
        #return self.code().unencode(self.decode_to_code(word))

Goppa._registered_decoders["GoppaDecoder"] = GoppaDecoder

# Niederreiter cryptosystem

In [115]:
def get_weight(c):
    w = 0
    
    for elem in c:
        if elem != 0:
            w += 1
    
    return w

In [134]:
class Niederreiter:
    r"""
    Implementation of Niederreiter cryptosystem.
    
    INPUT:
    - ``g`` -- a monic polynomial with coefficients in
    a finite fiel `\GF{p^q}`
    
    - ``defining_set`` -- tuple of n distinct elements of `\GF{p^m}`
    that are roots of `generating_pol
    """
    def __init__(self, n, p, q, g):
        """
        Initialize.
        """
        
        if not g.is_irreducible:
            raise ValueError("ERROR. Generating polynomial isn't irreducible")
        
        F = GF(2) # todo esto es asi?
        L = GF(2^q)
        
        defining_set = get_defining_set(n, g, L)

        C = codes.GoppaCode(g, defining_set)
        H = C.parity_check_matrix()
        k = H.nrows()
        n = H.ncols()
        
        print(C)
        
        """C = Goppa(defining_set, g, F)
        H = C.get_parity_check_matrix()
        k = H.nrows()
        n = H.ncols()"""
        
        # random binary nonsingular matrix
        S = matrix(F, k, [choice(F.list()) for i in range(k^2)])
        
        while rank(S) < k:
            i = randint(0, k-1)
            j = randint(0, k-1)
            S[i,j] = choice(F.list())
        
        # random permutation matrix
        columns = list(range(n))
        P = matrix(F, n)
        
        for i in range(n):
            l = randint(0, len(columns)-1)
            j = columns[l]
            P[i,j] = 1
            columns.remove(j)
            
        print("H:")
        print(H)
        
        self._goppa = C
        self._H = H
        self._t = g.degree()
        self._k = k
        self._n = n
        self._S = S
        self._P = P
        self._public_key = S * self._H * P
        
    def get_S(self):
        # TODO borrar
        return self._S
    
    def get_P(self):
        # TODO borrar
        return self._P
    
    def get_public_key(self):
        return self._public_key

    def encrypt(self, m):
        """
        Return a chipertext
        
        INPUT:
        - ``m``: a plaintext to encrypt
        """
        # encode
        # Borrar
        E = codes.encoders.GoppaCodeEncoder(self._goppa)
        #E = GoppaEncoder(self._goppa)
        m = E.encode(m)
        print("m encoded:")
        print(m)
        
        # codeword
        c = m * self._public_key.T
        
        return c
    
    def decrypt(self, c):
        print("c:")
        print(c)
        
        word = self._S^(-1) * c.column()
        
        print("word:")
        print(word)
        
        z = vector(self._H.solve_right(word).T)
        
        print("z:")
        print(z)
        
        D = self._goppa.decoder()
        word = D.decode_to_message(z)
        
        print("word")
        print(word)
        
        message = word * (self._S)^(-1)
        
        return message

In [135]:
p = 1 # siempre
q = 4
n = 15
t = floor((2+(2^q-1)/q)/2)

F = GF(2^p)
L = GF(2^q)
a = L.gen()
b = F.gen()
R.<x> = L[]

while True:
    g = R.random_element(t)
    
    if g.is_irreducible() and g.is_monic():
        break

print("Polinomio irreducible y mónico:")
print(g)

Polinomio irreducible y mónico:
x^2 + (z4^3 + z4^2 + z4 + 1)*x + z4^3 + z4 + 1


In [136]:
N = Niederreiter(n, p, q, g)

print("Matriz S:")
show(N.get_S())

print("Matriz P:")
#show(N.get_P())

print("Matriz G' (clave pública):")
show(N.get_public_key())

[15, 7] Goppa code over GF(2)
H:
[1 1 1 1 0 1 1 1 1 0 1 0 0 1 1]
[0 0 1 1 1 0 1 1 0 0 0 1 0 0 1]
[0 1 0 1 0 0 0 1 0 0 0 0 0 1 1]
[1 0 1 0 1 0 1 0 0 1 1 1 1 0 1]
[1 0 0 0 1 0 1 1 1 0 1 1 1 0 0]
[0 1 0 1 0 0 1 0 1 1 1 0 1 0 0]
[0 1 0 1 1 1 0 1 0 1 1 0 1 0 1]
[0 0 1 0 1 0 1 1 1 1 1 0 1 0 0]
Matriz S:


[1 0 0 0 0 1 0 1]
[0 1 1 1 1 0 0 1]
[1 1 0 1 1 0 0 1]
[1 0 0 1 0 1 1 1]
[1 0 0 0 0 1 0 0]
[0 0 0 1 0 1 0 0]
[1 1 1 1 0 0 1 1]
[0 1 1 0 0 1 1 0]

Matriz P:
Matriz G' (clave pública):


[0 1 0 1 1 0 0 0 1 1 0 0 1 1 1]
[1 0 0 0 1 1 1 0 0 1 1 0 0 0 1]
[1 1 0 1 1 1 0 0 1 1 1 0 0 1 0]
[0 0 1 0 1 1 1 0 0 1 1 1 1 1 1]
[1 1 1 1 1 0 1 1 0 1 0 0 0 0 0]
[0 1 0 0 0 1 1 0 0 1 1 1 1 1 0]
[1 0 1 0 0 0 0 1 0 1 1 0 0 0 0]
[0 0 1 1 1 1 1 0 0 1 1 0 0 1 0]

In [137]:
print("Public key:")
public_key = N.get_public_key()
show(public_key)

k = public_key.ncols() - public_key.nrows()

message = []
while len(message) != k:
    message = message + [choice(F.list())]

message = vector(F, message)

print("\nMessage to encrypt:")
print(message)

encrypted_message = N.encrypt(message)

print("\nEncrypted message:")
print(encrypted_message)

Public key:


[0 1 0 1 1 0 0 0 1 1 0 0 1 1 1]
[1 0 0 0 1 1 1 0 0 1 1 0 0 0 1]
[1 1 0 1 1 1 0 0 1 1 1 0 0 1 0]
[0 0 1 0 1 1 1 0 0 1 1 1 1 1 1]
[1 1 1 1 1 0 1 1 0 1 0 0 0 0 0]
[0 1 0 0 0 1 1 0 0 1 1 1 1 1 0]
[1 0 1 0 0 0 0 1 0 1 1 0 0 0 0]
[0 0 1 1 1 1 1 0 0 1 1 0 0 1 0]


Message to encrypt:
(1, 0, 1, 0, 0, 0, 1)
m encoded:
(1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1)

Encrypted message:
(0, 1, 0, 1, 0, 1, 1, 0)


In [138]:
message2 = N.decrypt(encrypted_message)

c:
(0, 1, 0, 1, 0, 1, 1, 0)
word:
[0]
[1]
[1]
[1]
[0]
[0]
[0]
[0]
z:
(0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0)
word
(0, 1, 0, 0, 1, 1, 0)


TypeError: unsupported operand parent(s) for *: 'Vector space of dimension 7 over Finite Field of size 2' and 'Full MatrixSpace of 8 by 8 dense matrices over Finite Field of size 2'

# Temporal

Notice that in Niederreiter’s scheme, the plaintext message is represented as the error of the codeword instead of the original information word. Therefore, the plain text message needs to be additionally encoded to a weight-t vector for encryption